In [1]:
F = GF(2)
V = VectorSpace(F, 3)
G = GL(3, GF(2))

In [2]:
points = [U for U in V.subspaces(1)]
len(points)

7

In [3]:
planes = [U for U in V.subspaces(2)]
len(planes)

7

In [4]:
point_index = {points[i]: i for i in range(len(points))}
plane_index = {planes[i]: i for i in range(len(planes))}

In [5]:
edges = []
for p in points:
    for H in planes:
        if p.is_subspace(H):
            edges.append((p, H))
len(edges)

21

In [6]:
edge_index = {edges[i]: i for i in range(len(edges))}

In [7]:
num_vertices = len(points) + len(planes)
num_edges = len(edges)

boundary = Matrix(CDF, num_vertices, num_edges)

In [8]:
for (p, H), j in edge_index.items():
    i_p = point_index[p]
    i_H = plane_index[H] + len(points)

    boundary[i_H, j] = 1     # +H
    boundary[i_p, j] = -1    # -p

In [9]:
ker = boundary.right_kernel()
ker.dimension()


8

In [10]:
ker

Vector space of degree 21 and dimension 8 over Complex Double Field
Basis matrix:
[ 1.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0 -1.0]
[ 0.0  1.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  0.0  1.0  1.0  0.0 -1.0]
[ 0.0  0.0  0.0  1.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  1.0  0.0  0.0  0.0  1.0  0.0 -1.0]
[ 0.0  0.0  0.0  0.0  1.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  1.0  1.0  0.0 -1.0]
[ 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0 -1.0  0.0  0.0  0.0 -1.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0 -1.0]
[ 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  1.0  0.0  1.0 -1.0]
[ 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0 -1.0  0.0  0.0  0.0 -1.0  0.0  1.0  0.0  1.0 -1.0]
[ 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0 -1.0  0.0 -1.0  1.0  0.0  0.0  0.0  0.0  1.0 -1.0]

In [11]:
steinberg_basis = ker.basis()
steinberg_basis[0]

(1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0)

In [12]:
def line_repr(L):
    """
    Canonical representative for a 1-dim subspace over F_2:
    return its unique nonzero vector.
    """
    for v in L.basis():
        if v != 0:
            return tuple(v)

In [13]:
def linear_form_str(a):
    """
    a is a vector in F_2^3, printed as a linear equation a·x = 0
    """
    terms = []
    for i, ai in enumerate(a):
        if ai != 0:
            terms.append(f"x{i+1}")
    if not terms:
        return "0"
    return " + ".join(terms)

In [14]:
def plane_repr(H):
    """
    Represent a plane as a readable equation like x1 + x2 = 0
    """
    V = H.ambient_vector_space()
    for a in V:
        if a != 0:
            if all(a * v == 0 for v in H.basis()):
                return "{" + linear_form_str(a) + " = 0}"

In [15]:
def edge_repr(edge):
    p, H = edge
    return f"{line_repr(p)} ⊂ {plane_repr(H)}"

In [16]:
def pretty_print_cycle(v, edges):
    for j, coeff in enumerate(v):
        if coeff != 0:
            print(f"{coeff:+} · [{edge_repr(edges[j])}]")


In [17]:
pretty_print_cycle(steinberg_basis[0],edges)

+1.0 · [(1, 0, 0) ⊂ {x3 = 0}]
-1.0 · [(1, 0, 0) ⊂ {x2 = 0}]
-1.0 · [(0, 1, 0) ⊂ {x3 = 0}]
+1.0 · [(0, 1, 0) ⊂ {x1 = 0}]
+1.0 · [(0, 0, 1) ⊂ {x2 = 0}]
-1.0 · [(0, 0, 1) ⊂ {x1 = 0}]


In [18]:
from itertools import permutations

def apartment_cycle_from_g(g, edges, edge_index):

    basis = [g.matrix().column(i) for i in range(3)]
    lines = [V.subspace([v]) for v in basis]

    cycle = vector(CDF, len(edges))

    for w in permutations([0, 1, 2]):
        sign = Permutation([i+1 for i in w]).signature()
        p = lines[w[0]]
        H = lines[w[0]] + lines[w[1]]
        j = edge_index[(p, H)]
        cycle[j] += sign

    return cycle

In [19]:
g = G.random_element()

In [20]:
v_A = apartment_cycle_from_g(g, edges, edge_index); v_A

(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0, 0.0, -1.0, 1.0, -1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [21]:
#verify this is actually a cycle by multiplying by boundary map
print((boundary * v_A).is_zero())

True


In [22]:
pretty_print_cycle(v_A,edges)

+1.0 · [(1, 1, 0) ⊂ {x3 = 0}]
-1.0 · [(1, 1, 0) ⊂ {x1 + x2 = 0}]
-1.0 · [(1, 1, 1) ⊂ {x1 + x3 = 0}]
+1.0 · [(1, 1, 1) ⊂ {x1 + x2 = 0}]
-1.0 · [(0, 1, 0) ⊂ {x3 = 0}]
+1.0 · [(0, 1, 0) ⊂ {x1 + x3 = 0}]


In [59]:
###### DEFINE G-ACTION ON STEINBERG REPRESENTATION #######

In [54]:
def g_act_on_subspace(g, S):
    """
    Given g in GL_3(F_2) and a subspace S of W,
    return g(S) as a subspace of W.
    """
    W = S.ambient_vector_space()
    basis = S.basis_matrix().transpose()
    transformed_columns = [g * basis.column(i) for i in range(basis.ncols())]
    vecs = [W(v) for v in transformed_columns]  # convert each to W-element explicitly
    return W.subspace(vecs)

In [ ]:
def g_action_on_edge_indices(g, edges, edge_index):
    """
    Returns a dictionary mapping edge indices under the action of g.
    """
    new_index_map = {}
    for i, (p, H) in enumerate(edges):
        gp = g_act_on_subspace(g, p)
        gH = g_act_on_subspace(g, H)
        j = edge_index[(gp, gH)]
        new_index_map[i] = j
    return new_index_map

In [56]:
def g_action_on_vector(g, v, edges, edge_index):
    perm_map = g_action_on_edge_indices(g, edges, edge_index)
    # create a zero vector of same dimension
    v_new = vector(CDF, len(v))
    for i, val in enumerate(v):
        if val != 0:
            j = perm_map[i]
            v_new[j] += val
    return v_new

In [58]:
v0 = steinberg_basis[0]; print(v0)
g_v0 = g_action_on_vector(g, v0, edges, edge_index); print(g_v0)

(1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0)
(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0, 0.0, -1.0, 1.0, -1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)


In [ ]:
# verify the action is in fact a G-action on H(\Delta; \C), the Steinberg representation

h = G.random_element()
lhs = g_action_on_vector(g, g_action_on_vector(h, v0, edges, edge_index), edges, edge_index)
rhs = g_action_on_vector(g * h, v0, edges, edge_index)
assert lhs == rhs, "G-action property failed!"